In [10]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 親ディレクトリのファイルをインポートするための設定
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from dataset.mnist import load_mnist # 親ディレクトリにmnistのデータを置いてください
from tqdm import tqdm

# 各種関数を定義しておく
def identity_function(x):
    return x


def step_function(x):
    return np.array(x > 0, dtype=np.int)


def sigmoid(x):
    return 1 / (1 + np.exp(-x))    


def sigmoid_grad(x):
    return (1.0 - sigmoid(x)) * sigmoid(x)
    

def relu(x):
    return np.maximum(0, x)


def relu_grad(x):
    grad = np.zeros(x)
    grad[x>=0] = 1
    return grad
    

def softmax(x):
    if x.ndim == 2:
        x = x.T
        x = x - np.max(x, axis=0)
        y = np.exp(x) / np.sum(np.exp(x), axis=0)
        return y.T 

    x = x - np.max(x) # オーバーフロー対策
    return np.exp(x) / np.sum(np.exp(x))


def sum_squared_error(y, t):
    return 0.5 * np.sum((y-t)**2)


def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    # 教師データがone-hot-vectorの場合、正解ラベルのインデックスに変換
    if t.size == y.size:
        t = t.argmax(axis=1)
             
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size


def softmax_loss(X, t):
    y = softmax(X)
    return cross_entropy_error(y, t)


#2 層のニューラルネットを構築するクラス
class TwoLayerNet:
    
    # 重み、バイアスを初期化する関数
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size) #重みは標準正規分布で初期化
        self.params['b1'] = np.zeros(hidden_size) #バイアスは0で初期化
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size) #重みは標準正規分布で初期化
        self.params['b2'] = np.zeros(output_size) #バイアスは0で初期化
    
    # 予測確率を返す関数
    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
    
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        
        return y
        
    # クロスエントロピー誤差を返す関数、x:入力データ, t:正解ラベル
    def loss(self, x, t):
        y = self.predict(x)
        
        return cross_entropy_error(y, t)
    
    # 正解率を返す関数
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
    
    # 数値微分で求めた傾きを返す関数
    def numerical_gradient(f, x):
        h = 1e-4 # 0.0001
        grad = np.zeros_like(x)
        
        it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
        while not it.finished:
            idx = it.multi_index
            tmp_val = x[idx]
            x[idx] = tmp_val + h
            fxh1 = f(x) # f(x+h)
            
            x[idx] = tmp_val - h 
            fxh2 = f(x) # f(x-h)
            grad[idx] = (fxh1 - fxh2) / (2*h)
            
            x[idx] = tmp_val # 値を元に戻す
            it.iternext()   
            
            return grad

    # 数値部分で求めた傾きを辞書に格納する関数、x:入力データ, t:正解ラベル
    def numerical_gradient2(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
    
    # 誤差逆伝播法で求めた傾きを返す関数
    def gradient(self, x, t):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        grads = {}
        
        batch_num = x.shape[0]
        
        # forward
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        
        # backward
        dy = (y - t) / batch_num
        grads['W2'] = np.dot(z1.T, dy)
        grads['b2'] = np.sum(dy, axis=0)
        
        dz1 = np.dot(dy, W2.T)
        da1 = sigmoid_grad(a1) * dz1
        grads['W1'] = np.dot(x.T, da1)
        grads['b1'] = np.sum(da1, axis=0)

        return grads
    
# MNIST画像データの読み込み
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000  # 繰り返しの回数を適宜設定する(16.7epoch)
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = [] #訓練データの交差エントロピー誤差を入れるリスト
train_acc_list = [] #訓練データの正解率を入れるリスト
test_acc_list = [] #テストデータの正解率を入れるリスト

# 1エポックあたりの繰り返し回数
iter_per_epoch = max(train_size / batch_size, 1)

# 10000回ミニバッチを回して学習させる
for i in tqdm(range(iters_num)):
    # ミニバッチの作成。60,000の画像から100枚選ぶ
    batch_mask = np.random.choice(train_size, batch_size,replace=False)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 勾配の計算(数値微分を使う)
    # grad = network.numerical_gradient2(x_batch, t_batch
    # 勾配の計算(誤差逆伝播法を使う)
    grad = network.gradient(x_batch, t_batch)
    
    # 重みとバイアスは1バッチごとに更新する（つまり、10000回更新する）
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    # 損失関数（交差エントロピー誤差）を計算する
    loss = network.loss(x_batch, t_batch)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        train_loss_list.append(loss)
        print(str(int(i/iter_per_epoch))+" epoch : "+" "+"交差エントロピー誤差, 訓練データ正解率, テストデータ正解率 | " + str(loss)+ ", "  + str(train_acc) + ", " + str(test_acc))

# グラフの描画
plt.style.use("ggplot")
# 余白を設定
fig=plt.figure(figsize=(15,5),dpi=200)
fig.subplots_adjust(hspace=0.6, wspace=0.2)
ax1=fig.add_subplot(121)
ax2=fig.add_subplot(122)

# 損失関数の推移
x = np.arange(len(train_loss_list))
ax1.plot(x, train_loss_list)
ax1.set_xlabel("epochs")
ax1.set_ylabel("loss")
ax1.set_ylim(0, 2.5)

# 正解率の推移
markers = {'train': 'o', 'test': 's'}
x = np.arange(len(train_acc_list))
ax2.plot(x, train_acc_list, label='train acc')
ax2.plot(x, test_acc_list, label='test acc', linestyle='--')
ax2.set_xlabel("epochs")
ax2.set_ylabel("accuracy")
ax2.set_ylim(0, 1.0)
ax2.legend(loc='lower right')


  0%|          | 0/10000 [00:00<?, ?it/s]

TypeError: numerical_gradient2() missing 2 required positional arguments: 'x' and 't'